# OpenAI API 실습

이 노트북에서는 OpenAI API 사용을 위한 환경 설정부터 실전 활용까지 모든 것을 다룹니다.

## 목차
1. 환경 설정
2. Tiktoken으로 토큰 이해하기
3. 기본 Chat Completions API
4. Streaming API
5. Embedding API
6. 모델 비교 및 선택 가이드
7. 실전 팁 (에러 처리, 비용 관리)
8. 미니 프로젝트: 간단한 챗봇 만들기

## 1. 환경 설정

### 1.1 Anaconda 설치 (처음 시작하는 경우)

**Windows/Mac/Linux 공통:**
1. [Anaconda 공식 사이트](https://www.anaconda.com/download) 접속
2. 운영체제에 맞는 설치 파일 다운로드
3. 설치 진행 (기본 설정으로 진행)

### 1.2 가상환경 생성

터미널(Anaconda Prompt)에서 실행:
```bash
# 가상환경 생성
conda create -n openai_env python=3.10 -y

# 가상환경 활성화
conda activate openai_env

# 주피터 노트북 설치
conda install jupyter -y
```

### 1.3 필수 라이브러리 설치

```bash
# OpenAI 공식 라이브러리
pip install openai>=1.55.0

# 토큰 카운팅 라이브러리
pip install tiktoken

# 환경변수 관리
pip install python-dotenv

# 웹 데모를 위한 라이브러리
pip install streamlit

# 데이터 처리 (옵션)
pip install numpy pandas
```

### 1.4 API 키 설정

1. [OpenAI Platform](https://platform.openai.com/api-keys)에서 API 키 발급
2. 프로젝트 루트에 `.env` 파일 생성
3. 다음 내용 입력:
```
OPENAI_API_KEY=sk-proj-your-api-key-here
```

⚠️ **중요:** `.env` 파일은 절대 Git에 커밋하지 마세요! `.gitignore`에 추가하세요.

In [2]:
# 설치된 라이브러리 버전 확인
import sys
print(f"Python 버전: {sys.version}")
import openai
import tiktoken
from dotenv import load_dotenv

Python 버전: 3.10.18 (main, Jun  5 2025, 08:37:47) [Clang 14.0.6 ]


In [3]:
# 환경변수 로드 및 클라이언트 초기화
import os
from openai import OpenAI
from dotenv import load_dotenv

# .env 파일에서 환경변수 로드
load_dotenv()

# OpenAI 클라이언트 초기화
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY")
)

# API 키 확인 (앞 5자리만 표시)
api_key = os.getenv("OPENAI_API_KEY")
if api_key:
    print(f"✅ API 키 로드 완료: {api_key[:5]}...")
else:
    print("❌ API 키를 찾을 수 없습니다. .env 파일을 확인하세요.")

✅ API 키 로드 완료: sk-pr...


## 2. Tiktoken으로 토큰 이해하기

토큰은 OpenAI API의 기본 단위입니다. API 사용료는 토큰 수로 계산되므로 이해가 필수적입니다!

### 토큰이란?
- 텍스트를 처리하는 기본 단위
- 한국어: 대략 1글자 ≈ 1-2 토큰
- 영어: 대략 1단어 ≈ 1-2 토큰
- 공백, 문장부호도 토큰으로 카운트됨

In [5]:
import tiktoken

# GPT-4/GPT-3.5 모델의 인코더 로드
encoding = tiktoken.encoding_for_model("gpt-4o-mini")

# 텍스트를 토큰으로 분해
text = "안녕하세요! OpenAI API를 배워보자구요!"
tokens = encoding.encode(text)

print(f"원본 텍스트: {text}")
print(f"토큰 수: {len(tokens)}")
print(f"토큰 ID: {tokens}")
print(f"\n각 토큰 분해:")
for token in tokens:
    print(f"  {token} -> '{encoding.decode([token])}'")

원본 텍스트: 안녕하세요! OpenAI API를 배워보자구요!
토큰 수: 14
토큰 ID: [14307, 171731, 0, 7788, 17527, 10328, 4831, 33628, 33771, 8122, 5947, 10997, 7952, 0]

각 토큰 분해:
  14307 -> '안'
  171731 -> '녕하세요'
  0 -> '!'
  7788 -> ' Open'
  17527 -> 'AI'
  10328 -> ' API'
  4831 -> '를'
  33628 -> ' 배'
  33771 -> '워'
  8122 -> '보'
  5947 -> '자'
  10997 -> '구'
  7952 -> '요'
  0 -> '!'


In [6]:
# 한국어 vs 영어 토큰 비교
texts = {
    "한국어": "인공지능은 미래의 핵심 기술입니다.",
    "영어": "Artificial intelligence is a key technology of the future.",
    "코드": "def hello_world():\n    print('Hello, World!')",
    "이모지": "안녕하세요! 😊 좋은 하루 되세요! 🌟"
}

print("=" * 60)
for lang, text in texts.items():
    tokens = encoding.encode(text)
    print(f"{lang}:")
    print(f"  텍스트: {text}")
    print(f"  글자 수: {len(text)}")
    print(f"  토큰 수: {len(tokens)}")
    print(f"  토큰/글자 비율: {len(tokens)/len(text):.2f}")
    print("-" * 60)

한국어:
  텍스트: 인공지능은 미래의 핵심 기술입니다.
  글자 수: 19
  토큰 수: 11
  토큰/글자 비율: 0.58
------------------------------------------------------------
영어:
  텍스트: Artificial intelligence is a key technology of the future.
  글자 수: 58
  토큰 수: 10
  토큰/글자 비율: 0.17
------------------------------------------------------------
코드:
  텍스트: def hello_world():
    print('Hello, World!')
  글자 수: 45
  토큰 수: 12
  토큰/글자 비율: 0.27
------------------------------------------------------------
이모지:
  텍스트: 안녕하세요! 😊 좋은 하루 되세요! 🌟
  글자 수: 21
  토큰 수: 11
  토큰/글자 비율: 0.52
------------------------------------------------------------


In [7]:
# 토큰 수로 비용 계산하기
def calculate_cost(text, model="gpt-4o-mini"):
    """텍스트의 예상 비용 계산 (2024년 10월 기준)"""
    encoding = tiktoken.encoding_for_model(model)
    tokens = len(encoding.encode(text))
    
    # 모델별 가격 (input 기준, 1M 토큰당 USD)
    prices = {
        "gpt-4o": 2.50,
        "gpt-4o-mini": 0.150,
        "gpt-3.5-turbo": 0.50,
    }
    
    price_per_token = prices.get(model, 0.150) / 1_000_000
    cost = tokens * price_per_token
    
    return {
        "tokens": tokens,
        "cost_usd": cost,
        "cost_krw": cost * 1300  # 환율 대략 1300원 가정
    }

# 예시
long_text = """인공지능(AI)은 현대 사회에서 가장 중요한 기술 중 하나로 자리잡고 있습니다. 
특히 자연어 처리 분야에서는 GPT와 같은 대규모 언어 모델이 혁신을 이끌고 있습니다.
이러한 모델들은 방대한 양의 텍스트 데이터를 학습하여 인간과 유사한 텍스트를 생성할 수 있습니다."""

result = calculate_cost(long_text, "gpt-4o-mini")
print(f"텍스트 길이: {len(long_text)} 글자")
print(f"토큰 수: {result['tokens']}")
print(f"예상 비용: ${result['cost_usd']:.6f} (약 {result['cost_krw']:.4f}원)")
print(f"\n💡 1만 글자 처리 시 예상 비용: 약 {result['cost_krw'] * 10000 / len(long_text):.2f}원")

텍스트 길이: 149 글자
토큰 수: 75
예상 비용: $0.000011 (약 0.0146원)

💡 1만 글자 처리 시 예상 비용: 약 0.98원


## 3. 기본 Chat Completions API

### API 기본 구조
```python
response = client.chat.completions.create(
    model="gpt-4o-mini",              # 필수: 사용할 모델
    messages=[...],                   # 필수: 대화 내용
    temperature=0.7,                  # 선택: 창의성 (0.0~2.0)
    max_tokens=150,                   # 선택: 최대 출력 토큰
    top_p=1.0,                       # 선택: 누적 확률
    frequency_penalty=0.0,            # 선택: 반복 억제 (0.0~2.0)
    presence_penalty=0.0,             # 선택: 주제 다양성 (0.0~2.0)
    stop=["\n", "###"],              # 선택: 생성 중단 조건
)
```

In [8]:
# 첫 번째 API 호출
try:
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "user", "content": "안녕하세요! 처음으로 API를 사용해봅니다."}
        ]
    )
    
    print("✅ API 호출 성공!\n")
    print("AI 응답:", response.choices[0].message.content)
    print(f"\n📊 사용 통계:")
    print(f"  - 입력 토큰: {response.usage.prompt_tokens}")
    print(f"  - 출력 토큰: {response.usage.completion_tokens}")
    print(f"  - 총 토큰: {response.usage.total_tokens}")
    
except Exception as e:
    print(f"❌ 오류 발생: {e}")

✅ API 호출 성공!

AI 응답: 안녕하세요! API를 처음 사용하신다니 흥미로운 경험이 될 것 같습니다. 어떤 API를 사용하고 계신가요? 또는 API 사용에 대해 궁금한 점이 있으신가요? 도움이 필요하시면 언제든지 말씀해 주세요!

📊 사용 통계:
  - 입력 토큰: 19
  - 출력 토큰: 57
  - 총 토큰: 76


### Messages 구조 이해하기

**3가지 역할(role):**
- `system`: AI의 행동 방식, Instruction 등 정의
- `user`: 사용자의 입력
- `assistant`: AI의 이전 응답 (대화 기록)

In [9]:
# System 메시지의 힘
def test_system_message(system_content, user_content):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_content},
            {"role": "user", "content": user_content}
        ],
        temperature=0.7
    )
    return response.choices[0].message.content

# 같은 질문, 다른 페르소나
question = "Python을 배우는 가장 좋은 방법은?"

print("=" * 60)
print("페르소나 1: 친절한 선생님")
print(test_system_message(
    "당신은 초보자를 위한 친절한 프로그래밍 선생님입니다. 쉽게 설명해주세요.",
    question
))

print("\n" + "=" * 60)
print("페르소나 2: 실전 개발자")
print(test_system_message(
    "당신은 10년차 시니어 개발자입니다. 실무 관점에서 핵심만 간단히 답변하세요.",
    question
))

페르소나 1: 친절한 선생님
Python을 배우는 가장 좋은 방법은 다음과 같은 단계로 진행하는 것입니다:

1. **기본 개념 이해하기**:
   - Python의 기본 문법과 개념(변수, 데이터 타입, 조건문, 반복문 등)을 배우세요. 이론을 배우는 것과 함께 간단한 예제를 작성해보는 것이 중요합니다.

2. **온라인 강좌 수강하기**:
   - Coursera, edX, Udemy와 같은 플랫폼에서 Python 강좌를 찾아 수강해 보세요. 초보자를 위한 강좌가 많이 있습니다.

3. **책 읽기**:
   - "Automate the Boring Stuff with Python" 같은 입문서나 "Python Crash Course"와 같은 책을 읽어보세요. 실습 예제가 많아서 도움이 됩니다.

4. **코드 작성하기**:
   - 직접 코드를 작성해보는 것이 가장 중요합니다. 간단한 프로그램을 만들어 보세요. 예를 들어, 계산기, 간단한 게임 등을 만들어 볼 수 있습니다.

5. **프로젝트 진행하기**:
   - 자신이 관심 있는 주제를 가지고 작은 프로젝트를 진행해 보세요. 예를 들어, 웹 스크래핑, 데이터 분석, 자동화 스크립트 등을 시도해 볼 수 있습니다.

6. **커뮤니티 참여하기**:
   - Stack Overflow, GitHub, Reddit의 Python 관련 커뮤니티에 참여해 보세요. 다른 사람들의 질문을 보고 답변하거나, 자신의 질문을 올려서 도움을 받을 수 있습니다.

7. **꾸준한 연습**:
   - 매일 조금씩이라도 코딩하는 습관을 들이면 좋습니다. 코딩 연습 사이트인 LeetCode, HackerRank, Codewars에서 문제를 풀어보세요.

8. **오픈소스 프로젝트 참여하기**:
   - GitHub에서 오픈소스 프로젝트에 기여해 보세요. 실제로 다른 사람들과 협업하면서 배우는 좋은 기회가 될 것입니다.

가장 중요한 것은 꾸준함과 호기심입니다. 처음에는 어렵게 느껴질 수 있지만, 계속해서 도전하다 보면 점점 더 

In [10]:
# 대화 기록 유지하기
conversation = [
    {"role": "system", "content": "당신은 도움이 되는 AI 어시스턴트입니다."},
]

def chat(user_message):
    """대화 기록을 유지하며 채팅"""
    # 사용자 메시지 추가
    conversation.append({"role": "user", "content": user_message})
    
    # API 호출
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=conversation,
        temperature=0.7
    )
    
    # AI 응답 추가
    ai_message = response.choices[0].message.content
    conversation.append({"role": "assistant", "content": ai_message})
    
    return ai_message

# 연속 대화 테스트
print("User: 파이썬으로 리스트를 정렬하는 방법을 알려줘")
print(f"AI: {chat('파이썬으로 리스트를 정렬하는 방법을 알려줘')}\n")

print("User: 역순으로는 어떻게 해?")
print(f"AI: {chat('역순으로는 어떻게 해?')}\n")

print(f"\n현재 대화 길이: {len(conversation)} 메시지")

User: 파이썬으로 리스트를 정렬하는 방법을 알려줘
AI: 파이썬에서 리스트를 정렬하는 방법은 여러 가지가 있습니다. 가장 일반적인 두 가지 방법은 `sort()` 메서드와 `sorted()` 함수를 사용하는 것입니다.

1. **`sort()` 메서드**:
   - `sort()`는 리스트 객체의 메서드로, 리스트 자체를 정렬합니다. 이 메서드는 리스트를 직접 수정하므로 반환값은 `None`입니다.

   ```python
   my_list = [5, 2, 9, 1, 5, 6]
   my_list.sort()  # 리스트를 오름차순으로 정렬
   print(my_list)   # 출력: [1, 2, 5, 5, 6, 9]
   ```

   - 내림차순으로 정렬하려면 `reverse=True` 인자를 사용할 수 있습니다.

   ```python
   my_list.sort(reverse=True)
   print(my_list)   # 출력: [9, 6, 5, 5, 2, 1]
   ```

2. **`sorted()` 함수**:
   - `sorted()` 함수는 iterable(반복 가능한 객체)을 인자로 받아 정렬된 새로운 리스트를 반환합니다. 원래 리스트는 변경되지 않습니다.

   ```python
   my_list = [5, 2, 9, 1, 5, 6]
   sorted_list = sorted(my_list)  # 새로운 정렬된 리스트 생성
   print(sorted_list)  # 출력: [1, 2, 5, 5, 6, 9]
   print(my_list)      # 원래 리스트는 그대로: [5, 2, 9, 1, 5, 6]
   ```

   - 마찬가지로 내림차순으로 정렬할 수도 있습니다.

   ```python
   sorted_list_desc = sorted(my_list, reverse=True)
   print(sorted_list_desc)  # 출력: [9, 6, 5, 5, 2, 1]
   ```

3. **정렬 기준 지정하기

### Temperature - 창의성 조절

- `0.0`: 매우 결정적, 일관성 있는 답변
- `0.3-0.5`: 사실 기반 답변에 적합
- `0.7-0.9`: 균형잡힌 창의성 (기본값)
- `1.0-2.0`: 매우 창의적, 예측 불가능

In [11]:
prompt = "우주 여행을 주제로 한 단편 소설의 첫 문장을 써줘"

temperatures = [0.0, 0.5, 1.0, 1.5]

for temp in temperatures:
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=temp,
        max_tokens=100
    )
    print(f"\n🌡️ Temperature {temp}:")
    print(response.choices[0].message.content)
    print("-" * 60)


🌡️ Temperature 0.0:
우주 정거장에서 바라본 지구는 마치 푸른 보석처럼 반짝이며, 그 아래에서 잊혀진 기억들이 소용돌이치는 듯했다.
------------------------------------------------------------

🌡️ Temperature 0.5:
우주를 가로지르는 은하의 어둠 속에서, 인류는 처음으로 별빛을 따라 나아가는 여행을 시작했다.
------------------------------------------------------------

🌡️ Temperature 1.0:
푸른 지구를 뒤로하고, 그녀의 우주선은 은하의 어둠 속으로 조용히 스며들어 갔다.
------------------------------------------------------------

🌡️ Temperature 1.5:
먼 우주 이상에서 수없이 반짝이는 별들 사이로, 지구에서 로켓에 몸을 실은 한 소년은 한 편의 꿈을 안고 출발했다.
------------------------------------------------------------


## 4. Streaming API - 실시간 응답

일반 API는 모든 답변이 완성될 때까지 기다려야 하지만, Streaming을 사용하면 ChatGPT처럼 실시간으로 텍스트가 생성됩니다!

In [12]:
# 기본 스트리밍
print("AI: ", end="", flush=True)

stream = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "인공지능의 미래에 대해 3문장으로 설명해줘"}],
    stream=True,  # 스트리밍 활성화
)

for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="", flush=True)

print()  # 줄바꿈

AI: 인공지능은 다양한 산업과 일상생활에서 혁신적인 변화를 이끌 것으로 기대됩니다. 특히, 자동화와 데이터 분석을 통해 인간의 생산성을 높이고, 보다 개인화된 서비스와 솔루션을 제공할 수 있는 가능성이 큽니다. 그러나 윤리적 문제와 일자리 변화와 같은 도전 과제가 동반될 것이며, 이에 대한 사회적 논의가 필요합니다.


In [13]:
import time

def stream_chat(prompt, show_stats=True):
    """통계와 함께 스트리밍 채팅"""
    start_time = time.time()
    full_response = ""
    
    stream = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        stream=True,
    )
    
    print("AI: ", end="", flush=True)
    
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            content = chunk.choices[0].delta.content
            print(content, end="", flush=True)
            full_response += content
    
    elapsed_time = time.time() - start_time
    
    if show_stats:
        encoding = tiktoken.encoding_for_model("gpt-4o-mini")
        tokens = len(encoding.encode(full_response))
        print(f"\n\n📊 통계:")
        print(f"  - 응답 길이: {len(full_response)} 글자")
        print(f"  - 토큰 수: {tokens}")
        print(f"  - 소요 시간: {elapsed_time:.2f}초")
        print(f"  - 처리 속도: {tokens/elapsed_time:.1f} 토큰/초")
    
    return full_response

# 테스트
stream_chat("Python에서 리스트 컴프리헨션을 설명하고 3가지 예제를 보여줘")

AI: 리스트 컴프리헨션(List Comprehension)은 파이썬에서 리스트를 간결하게 생성할 수 있는 방법입니다. 전통적인 루프를 사용하는 것보다 더 간단하고 직관적으로 리스트를 생성할 수 있게 해줍니다. 기본적인 구문은 다음과 같습니다.

```python
[식 for 항목 in iterable if 조건]
```

이 구문은 `iterable`에서 각 `항목`을 반복(iterate)하면서 특정 `조건`을 만족하는 경우에만 `식`에 의해 생성된 값을 포함하는 리스트를 생성합니다.

### 예제 1: 제곱 리스트 만들기

1부터 10까지의 정수의 제곱을 포함하는 리스트를 생성합니다.

```python
squares = [x**2 for x in range(1, 11)]
print(squares)  # 출력: [1, 4, 9, 16, 25, 36, 49, 64, 81, 100]
```

### 예제 2: 짝수 필터링

1부터 20까지의 숫자 중에서 짝수만 포함하는 리스트를 생성합니다.

```python
evens = [x for x in range(1, 21) if x % 2 == 0]
print(evens)  # 출력: [2, 4, 6, 8, 10, 12, 14, 16, 18, 20]
```

### 예제 3: 문자열 길이 리스트

주어진 문자열 목록에서 각 문자열의 길이를 포함하는 리스트를 생성합니다.

```python
strings = ["apple", "banana", "cherry"]
lengths = [len(s) for s in strings]
print(lengths)  # 출력: [5, 6, 6]
```

리스트 컴프리헨션을 사용하면 코드가 더 간결하고 가독성이 좋아지는 장점이 있습니다. 이러한 방식은 파이썬의 "한 줄로 해결하는" 접근 방식에 잘 어울립니다.

📊 통계:
  - 응답 길이: 896 글자
  - 토큰 수: 444
  - 소요 시간: 8.58초
  - 처리 속도: 51.8 토큰/초


'리스트 컴프리헨션(List Comprehension)은 파이썬에서 리스트를 간결하게 생성할 수 있는 방법입니다. 전통적인 루프를 사용하는 것보다 더 간단하고 직관적으로 리스트를 생성할 수 있게 해줍니다. 기본적인 구문은 다음과 같습니다.\n\n```python\n[식 for 항목 in iterable if 조건]\n```\n\n이 구문은 `iterable`에서 각 `항목`을 반복(iterate)하면서 특정 `조건`을 만족하는 경우에만 `식`에 의해 생성된 값을 포함하는 리스트를 생성합니다.\n\n### 예제 1: 제곱 리스트 만들기\n\n1부터 10까지의 정수의 제곱을 포함하는 리스트를 생성합니다.\n\n```python\nsquares = [x**2 for x in range(1, 11)]\nprint(squares)  # 출력: [1, 4, 9, 16, 25, 36, 49, 64, 81, 100]\n```\n\n### 예제 2: 짝수 필터링\n\n1부터 20까지의 숫자 중에서 짝수만 포함하는 리스트를 생성합니다.\n\n```python\nevens = [x for x in range(1, 21) if x % 2 == 0]\nprint(evens)  # 출력: [2, 4, 6, 8, 10, 12, 14, 16, 18, 20]\n```\n\n### 예제 3: 문자열 길이 리스트\n\n주어진 문자열 목록에서 각 문자열의 길이를 포함하는 리스트를 생성합니다.\n\n```python\nstrings = ["apple", "banana", "cherry"]\nlengths = [len(s) for s in strings]\nprint(lengths)  # 출력: [5, 6, 6]\n```\n\n리스트 컴프리헨션을 사용하면 코드가 더 간결하고 가독성이 좋아지는 장점이 있습니다. 이러한 방식은 파이썬의 "한 줄로 해결하는" 접근 방식에 잘 어울립니다.'

In [14]:
# 일반 vs 스트리밍 체감 비교
prompt = "인공지능의 역사를 5문단으로 작성해줘"

print("=" * 60)
print("1️⃣ 일반 API (전체 응답 대기)\n")
start = time.time()
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt}],
)
print(f"⏱️ 첫 응답까지: {time.time() - start:.2f}초")
print(response.choices[0].message.content[:100] + "...")

print("\n" + "=" * 60)
print("2️⃣ 스트리밍 API (실시간 생성)\n")
start = time.time()
first_chunk_time = None

stream = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt}],
    stream=True,
)

for i, chunk in enumerate(stream):
    if chunk.choices[0].delta.content is not None:
        if first_chunk_time is None:
            first_chunk_time = time.time() - start
        if i < 50:  # 처음 일부만 출력
            print(chunk.choices[0].delta.content, end="", flush=True)

print(f"\n\n⏱️ 첫 응답까지: {first_chunk_time:.2f}초 (훨씬 빠름!)")

1️⃣ 일반 API (전체 응답 대기)

⏱️ 첫 응답까지: 12.15초
인공지능(AI)의 역사는 20세기 중반으로 거슬러 올라갑니다. 1956년 다트머스 회의에서 존 매카시, 마빈 민스키, 네이선 미르볼드 등이 모여 AI라는 용어를 처음 제안했습니다....

2️⃣ 스트리밍 API (실시간 생성)

인공지능(AI)의 역사는 20세기 중반으로 거슬러 올라갑니다. 1956년 다트머스 대학에서 열린 AI 여름 워크숍은 인공지능 연구의 시작을 알리는

⏱️ 첫 응답까지: 0.62초 (훨씬 빠름!)


## 5. Embedding API - 텍스트를 벡터로

Embedding은 텍스트를 숫자 벡터로 변환합니다. 이를 통해:
- 텍스트 유사도 계산
- 의미적 검색 (Semantic Search)
- 클러스터링, 분류

**모델:** `text-embedding-3-small` (저렴), `text-embedding-3-large` (정확)

In [15]:
# 기본 임베딩
def get_embedding(text, model="text-embedding-3-small"):
    """텍스트를 벡터로 변환"""
    text = text.replace("\n", " ")
    response = client.embeddings.create(
        input=[text],
        model=model
    )
    return response.data[0].embedding

# 예시
text = "인공지능은 미래의 핵심 기술입니다."
embedding = get_embedding(text)

print(f"텍스트: {text}")
print(f"벡터 차원: {len(embedding)}")
print(f"벡터 앞부분: {embedding[:10]}")
print(f"\n💡 이 벡터는 텍스트의 의미를 숫자로 표현한 것입니다!")

텍스트: 인공지능은 미래의 핵심 기술입니다.
벡터 차원: 1536
벡터 앞부분: [-0.01845400594174862, 0.030923428013920784, -0.02049209736287594, 0.005952158011496067, 0.013340245001018047, -0.04431925714015961, -0.004465276375412941, 0.04461570829153061, -0.0016061562346294522, -0.006114278919994831]

💡 이 벡터는 텍스트의 의미를 숫자로 표현한 것입니다!


In [16]:
import numpy as np

def cosine_similarity(vec1, vec2):
    """코사인 유사도 계산"""
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

# 여러 문장의 유사도 비교
sentences = [
    "강아지가 공원에서 뛰어놀고 있다",
    "개가 야외에서 놀고 있다",
    "고양이가 집에서 자고 있다",
    "Python은 프로그래밍 언어이다",
]

# 모든 문장을 임베딩으로 변환
embeddings = [get_embedding(s) for s in sentences]

# 첫 번째 문장과 나머지 문장들의 유사도
base_sentence = sentences[0]
print(f"기준 문장: '{base_sentence}'\n")
print("유사도 점수 (1에 가까울수록 유사):")

for i, sentence in enumerate(sentences):
    similarity = cosine_similarity(embeddings[0], embeddings[i])
    print(f"  {similarity:.4f} - {sentence}")

기준 문장: '강아지가 공원에서 뛰어놀고 있다'

유사도 점수 (1에 가까울수록 유사):
  1.0000 - 강아지가 공원에서 뛰어놀고 있다
  0.3027 - 개가 야외에서 놀고 있다
  0.4248 - 고양이가 집에서 자고 있다
  0.1319 - Python은 프로그래밍 언어이다


In [17]:
# 실전 예제: 간단한 의미 검색 시스템
documents = [
    "Python은 간결하고 배우기 쉬운 프로그래밍 언어입니다",
    "머신러닝은 데이터로부터 패턴을 학습하는 기술입니다",
    "딥러닝은 인공신경망을 사용하는 머신러닝의 한 분야입니다",
    "자연어 처리는 컴퓨터가 인간의 언어를 이해하게 하는 기술입니다",
    "JavaScript는 웹 개발에 주로 사용되는 언어입니다",
]

# 모든 문서 임베딩
doc_embeddings = [get_embedding(doc) for doc in documents]

def search(query, top_k=3):
    """의미 기반 검색"""
    query_embedding = get_embedding(query)
    
    # 유사도 계산
    similarities = [
        cosine_similarity(query_embedding, doc_emb) 
        for doc_emb in doc_embeddings
    ]
    
    # 상위 k개 결과
    results = sorted(
        zip(documents, similarities), 
        key=lambda x: x[1], 
        reverse=True
    )[:top_k]
    
    return results

# 검색 테스트
queries = [
    "AI 학습 방법",
    "프로그래밍 배우기",
    "언어 이해 기술"
]

for query in queries:
    print(f"\n🔍 검색어: '{query}'")
    results = search(query)
    for doc, score in results:
        print(f"  [{score:.4f}] {doc}")


🔍 검색어: 'AI 학습 방법'
  [0.3559] 머신러닝은 데이터로부터 패턴을 학습하는 기술입니다
  [0.3175] Python은 간결하고 배우기 쉬운 프로그래밍 언어입니다
  [0.3095] 자연어 처리는 컴퓨터가 인간의 언어를 이해하게 하는 기술입니다

🔍 검색어: '프로그래밍 배우기'
  [0.4961] Python은 간결하고 배우기 쉬운 프로그래밍 언어입니다
  [0.3243] JavaScript는 웹 개발에 주로 사용되는 언어입니다
  [0.2123] 머신러닝은 데이터로부터 패턴을 학습하는 기술입니다

🔍 검색어: '언어 이해 기술'
  [0.6288] 자연어 처리는 컴퓨터가 인간의 언어를 이해하게 하는 기술입니다
  [0.3627] Python은 간결하고 배우기 쉬운 프로그래밍 언어입니다
  [0.3366] JavaScript는 웹 개발에 주로 사용되는 언어입니다


## 6. 모델 비교 및 선택 가이드

### 주요 모델들 (2024년 10월 기준)

| 모델 | 특징 | 입력 가격* | 출력 가격* | 추천 용도 |
|------|------|-----------|-----------|----------|
| **gpt-4o** | 최신, 최고 성능 | $2.50 | $10.00 | 복잡한 추론, 전문 작업 |
| **gpt-4o-mini** | 가성비 최고 | $0.150 | $0.600 | 대부분의 일반 작업 |
| **gpt-3.5-turbo** | 빠르고 저렴 | $0.50 | $1.50 | 간단한 작업, 대화 |

*가격: 1M 토큰당 USD

In [18]:
# 같은 질문으로 모델 비교
prompt = """다음 텍스트의 감정을 분석하고 긍정/부정/중립으로 분류해주세요.
그리고 그렇게 판단한 이유를 간단히 설명해주세요.

텍스트: "제품 배송은 빨랐지만, 품질이 기대에 못 미쳤습니다. 가격 대비 아쉬운 점이 많네요."
"""

models = ["gpt-4o-mini", "gpt-4o", "gpt-3.5-turbo"]

for model in models:
    print(f"\n{'='*60}")
    print(f"모델: {model}")
    print(f"{'='*60}")
    
    start_time = time.time()
    
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
    )
    
    elapsed = time.time() - start_time
    
    print(response.choices[0].message.content)
    print(f"\n⏱️ 응답 시간: {elapsed:.2f}초")
    print(f"📊 토큰 사용: {response.usage.total_tokens}")


모델: gpt-4o-mini
감정 분석 결과: **부정**

이유: 텍스트는 제품 배송이 빨랐다는 긍정적인 요소로 시작하지만, 이어서 품질이 기대에 미치지 못하고 가격 대비 아쉬운 점이 많다는 부정적인 평가가 포함되어 있습니다. 전반적으로 부정적인 감정이 지배적이므로 부정으로 분류했습니다.

⏱️ 응답 시간: 4.93초
📊 토큰 사용: 158

모델: gpt-4o
이 텍스트의 감정은 부정으로 분류할 수 있습니다. 

이유:
1. "제품 배송은 빨랐지만"이라는 표현에서 배송 속도에 대한 긍정적인 부분이 언급되었으나, 이는 전체적인 평가에 큰 영향을 미치지 않습니다.
2. "품질이 기대에 못 미쳤습니다"라는 문장에서 제품의 품질에 대한 실망감이 드러납니다.
3. "가격 대비 아쉬운 점이 많네요"라는 표현은 가격에 비해 만족스럽지 않다는 부정적인 평가를 나타냅니다.

이러한 이유들로 인해 전체적인 감정은 부정적이라고 판단할 수 있습니다.

⏱️ 응답 시간: 3.58초
📊 토큰 사용: 225

모델: gpt-3.5-turbo
부정

이유: 텍스트에서는 제품의 품질이 기대에 못 미치고, 가격 대비 아쉬운 점이 많다는 내용이 담겨 있습니다. 이는 제품에 대한 실망과 아쉬움을 표현한 것으로 부정적인 감정을 나타내고 있습니다.

⏱️ 응답 시간: 1.85초
📊 토큰 사용: 233


### JSON 모드 - 구조화된 출력

In [19]:
import json

# JSON 형식으로 출력 받기
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system", 
            "content": "당신은 데이터를 JSON 형식으로 반환하는 어시스턴트입니다."
        },
        {
            "role": "user", 
            "content": """다음 텍스트에서 정보를 추출하여 JSON으로 반환해주세요:
            
            "홍길동은 30세 남성이며, 서울에 거주합니다. 직업은 소프트웨어 엔지니어이고, 
            취미는 독서와 등산입니다."
            
            JSON 형식: {"name": str, "age": int, "gender": str, "city": str, 
                        "job": str, "hobbies": list}
            """
        }
    ],
    response_format={"type": "json_object"},  # JSON 모드 활성화
    temperature=0.3,
)

result = json.loads(response.choices[0].message.content)
print("추출된 정보:")
print(json.dumps(result, ensure_ascii=False, indent=2))

추출된 정보:
{
  "name": "홍길동",
  "age": 30,
  "gender": "남성",
  "city": "서울",
  "job": "소프트웨어 엔지니어",
  "hobbies": [
    "독서",
    "등산"
  ]
}


## 8. 미니 프로젝트: 간단한 챗봇 만들기

지금까지 배운 내용을 종합하여 Streamlit을 활용한 실용적인 챗봇을 만들어봅시다!  

`Streamlit`은 쥬피터 노트북에서는 실행이 불가능해서, `streamlit_chatgpt.py` 형태로 코드를 뒀습니다.  
각자 터미널을 열고 아래처럼 실행해보세요:

```
cd ~/metacode-202014/01.ChatGPT-API
streamlit run streamlit_chatgpt.py
```